# Kafka: Produce and Consume

Test Kafka to produce and consume messages. Please refer to [Confluent's Apache Kafka Python client](https://github.com/confluentinc/confluent-kafka-python) for more details.

## License

MIT License

Copyright (c) 2018 PT Bukalapak.com

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Software Version

In [1]:
import sys
print("Python %s" % sys.version)

Python 3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]


In [2]:
import confluent_kafka
print("confluent_kafka %s" % confluent_kafka.__version__)
from confluent_kafka import Producer, Consumer, KafkaError

confluent_kafka 0.11.6


In [3]:
%%bash
cat /etc/os-release

NAME="Ubuntu"
VERSION="18.04.1 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.1 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


## Produce

In [4]:
broker = 'localhost'

p = Producer({'bootstrap.servers': broker})

In [5]:
def delivery_report(err, msg):
    # Called once for each message produced to indicate delivery result.
    # Triggered by poll() or flush().
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

In [6]:
some_data_source = [
    "hallo 1", 
    "hallo 2", 
    "hallo 3"
]

In [7]:
topic = 'dummy_topic_1'

for data in some_data_source:
    # Trigger any available delivery report callbacks from previous produce() calls
    p.poll(0)

    # Asynchronously produce a message, the delivery report callback
    # will be triggered from poll() above, or flush() below, when the message has
    # been successfully delivered or failed permanently.
    p.produce(topic, data.encode('utf-8'), callback=delivery_report)

In [8]:
# Wait for any outstanding messages to be delivered and delivery report
# callbacks to be triggered.
p.flush()

Message delivered to dummy_topic_1 [0]
Message delivered to dummy_topic_1 [0]
Message delivered to dummy_topic_1 [0]


0

## Consume

In [9]:
group = 'dummy_group_1'

c = Consumer({
    'bootstrap.servers': broker,
    'group.id': group,
    'default.topic.config': {
        'auto.offset.reset': 'smallest'
    }
})

In [10]:
c.subscribe(['dummy_topic_1'])

In [11]:
produced_len = len(some_data_source)

In [12]:
produced_len

3

In [13]:
for _ in range(produced_len):
    msg = c.poll(1.0)

    if msg is None:
        continue
    if msg.error():
        if msg.error().code() == KafkaError._PARTITION_EOF:
            continue
        else:
            print(msg.error())
            break

    print('Received message: {}'.format(msg.value().decode('utf-8')))

Received message: hallo 1
Received message: hallo 2
Received message: hallo 3


In [14]:
c.close()